# Planned Team with feeback loop

Like `planned_team.ipynb`, but with a feedback callback to repeat planning and execution until certain criteria are met. The idea is to instruct an `Agent` to be always last in the plan and fill in some variables with the feedback from the previous execution, if any. When `repeat_until` callback is invoked, it checks those variables and decides whether to repeat the planning and execution or not. `PlannedTeam` will then be able to plan and execute according to the given feedback.

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.llm import AzureOpenAILLM, LLMConstraints
from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.planned_team import PlannedTeam
from vanilla_aiagents.conversation import Conversation
from dotenv import load_dotenv

load_dotenv(override=True)
import os

In [62]:
llm = AzureOpenAILLM({
    "azure_deployment": "gpt-4o",
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})
reasoning_llm = AzureOpenAILLM({
    "azure_deployment": "o1-mini",
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
}, constraints=LLMConstraints(temperature=1, structured_output=False, system_message=False))

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.planned_team").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.user").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS


In [ ]:
# A sales agent that provides offers to users
sales = Agent(
    id="sales",
    llm=llm,
    description="Sales agent, takes price and discounts to provide an offer. Use a professional tone",
    system_message="""You are part of an AI sales process
Your task is to respond to the user buying ask by providing a price and a discount.
You're goal is to maximize sold value, so keep discount low unless the user keeps asking for it.
NEVER exceed the maximum discount for the product.
""",
)


DEBUG:vanilla_aiagents.agent:Agent initialized with ID: sales, Description: Sales agent, takes price and discounts to provide an offer. Use a professional tone


In [ ]:

# A catalog agent that provides prices and discounts for products
catalog = Agent(
    id="catalog",
    llm=llm,
    description="Product catalog agent, provides prices and discounts",
    system_message="""You are part of an AI process
Your task is to provide price and discount information for the sales agent to use in the offer.

# PRODUCTS PRICES AND DISCOUNTS
- Oven: $1000
    - MAX Discount: 25%
- Fridge: $1500
    - MAX Discount: 10%
- Washing machine: $800
    - MAX Discount: 15%
    
Use the following JSON format to provide the information:
{{"product": "name", "price": "1000", "max_discount": 0.10}}
""",
)


DEBUG:vanilla_aiagents.agent:Agent initialized with ID: catalog, Description: Product catalog agent, provides prices and discounts


In [ ]:
# A buyer agent that provides feedback on offers
# Key points: 
# - it is specified to call this last and to never accept the first offer, so at least one more execution must be done
# - agent instructions indicated which variables to set when accepting or rejecting the offer
buyer = Agent(
    id="buyer",
    llm=llm,
    description="Buyer agent, will provide feedback on the offer. MUST be called last in the workflow",
    system_message="""You are part of an AI sales process.
    Your task is to provide feedback on the offer provided by the sales agent.
    Never accept the first offer
    Minimum acceptable discount is 10%.
    
    When offer can be accepted, set variable "result" to "done"
    When offer is not acceptable, set variable "result" to "KO" and "__feedback" to "I want a better discount"
    """,
)


DEBUG:vanilla_aiagents.agent:Agent initialized with ID: buyer, Description: Buyer agent, will provide feedback on the offer. MUST be called last in the workflow


In [ ]:
# Callback logic, check the aforementioned variables to determine if the conversation can end
# Also, limit the conversation to 10 messages to avoid potentially infinite loops
def can_end(conversation: Conversation) -> bool:
    return conversation.variables.get("result") == "done" or len(conversation.messages) > 10

flow = PlannedTeam(
    id="offer",
    description="",
    members=[sales, catalog, buyer],
    # We use the reasoning_llm to provide the instructions to the agents, not required but useful
    llm=reasoning_llm,
    # Not mandatory, but useful to improve planning in general
    include_tools_descriptions=True,
    # Callback specified here
    repeat_until=can_end,
)
workflow = Workflow(askable=flow)

DEBUG:vanilla_aiagents.planned_team:[PlannedTeam offer] initialized with agents: {'sales': <vanilla_aiagents.agent.Agent object at 0x000001D9E87063D0>, 'catalog': <vanilla_aiagents.agent.Agent object at 0x000001D9E86BA2D0>, 'buyer': <vanilla_aiagents.agent.Agent object at 0x000001D9E853ECD0>}
DEBUG:vanilla_aiagents.workflow:Workflow initialized
DEBUG:vanilla_aiagents.workflow:Workflow initialized


In [67]:
workflow.restart()

workflow.run("I want a new oven")

DEBUG:vanilla_aiagents.workflow:Conversation length: 0
DEBUG:vanilla_aiagents.workflow:Restarted workflow, cleared conversation.
DEBUG:vanilla_aiagents.workflow:Running workflow with input: I want a new oven
DEBUG:vanilla_aiagents.workflow:Conversation length: 0
DEBUG:vanilla_aiagents.workflow:Added system prompt to messages: 
DEBUG:vanilla_aiagents.workflow:Added user input to messages: I want a new oven
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Windows-10-10.0.26100-SP0)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:httpx:HTTP Request: POST https://ricchi-oai-sw.openai.azure.com//openai/deployments/o1-mini/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 20

'done'

In [68]:
flow.plan

[TeamPlanStep(agent_id='catalog', instructions='Provide prices and available discounts for ovens.'),
 TeamPlanStep(agent_id='sales', instructions='Use the provided prices and discounts to create a professional offer for the new oven.'),
 TeamPlanStep(agent_id='buyer', instructions='Provide feedback on the offer for the new oven.')]

In [69]:
workflow.conversation.messages

[{'role': 'system', 'content': ''},
 {'role': 'user', 'name': 'user', 'content': 'I want a new oven'},
 {'role': 'assistant',
  'name': 'offer',
  'content': 'Provide prices and available discounts for ovens.'},
 {'content': 'Here is the price and discount information for an oven:\n\n```json\n{\n  "product": "oven",\n  "price": "1000",\n  "max_discount": 0.25\n}\n```',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'name': 'catalog'},
 {'role': 'assistant',
  'name': 'offer',
  'content': 'Use the provided prices and discounts to create a professional offer for the new oven.'},
 {'content': "The price of a new oven is $1,000. I can offer you a 10% discount, bringing the price down to $900. Let me know if you're ready to proceed or if you'd like to discuss further!",
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'name': 'sales'},
 {'role': 'assistant',
  'name': 'of